In [30]:
import pandas as pd

### update on 7/31/2024: make sure the earliest project on a parcel has "build" action

In [31]:
# pipeline data used in Draft Blueprint modeling
pipeline_dbp_version = pd.read_csv(
    r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\parcels_buildings_agents\development_pipeline_NP_2024-03-08.csv'
)
print('row count: {}'.format(len(pipeline_dbp_version)))
print('\nunique development_projects_id: {}\n'.format(pipeline_dbp_version['development_projects_id'].nunique()))
print(pipeline_dbp_version['action'].value_counts(dropna=False), '\n')
print(pipeline_dbp_version[['non_residential_sqft', 'residential_units']].sum(), '\n')
print(pipeline_dbp_version.groupby('action')[['non_residential_sqft', 'residential_units']].sum())

row count: 58723

unique development_projects_id: 58723

build    53713
add       5010
Name: action, dtype: int64 

non_residential_sqft    392460056.0
residential_units          439465.0
dtype: float64 

        non_residential_sqft  residential_units
action                                         
add               82823884.0            58031.0
build            309636172.0           381434.0


C:\Users\ywang\AppData\Local\Temp\ipykernel_12516\1866414638.py:2: DtypeWarning: Columns (3,4,6,7,8,9,14,22,23,26,27,34) have mixed types. Specify dtype option on import or set low_memory=False.
  pipeline_dbp_version = pd.read_csv(


In [32]:
# a function to modify the project list to ensure the earliest project on a parcel has "build" action

def ensure_earliest_proj_build(df):
    df = df.sort_values(["PARCEL_ID", "year_built"])
    prev_parcel_id = None
    for index, rec in df.iterrows():
        if rec.PARCEL_ID != prev_parcel_id:
            df.loc[index, "action"] = "build"
        prev_parcel_id = rec.PARCEL_ID
    
    return df

In [33]:
# apply the modification
pipeline_updated = ensure_earliest_proj_build(pipeline_dbp_version)

print(pipeline_updated['action'].value_counts(dropna=False), '\n')
print(pipeline_updated[['non_residential_sqft', 'residential_units']].sum(), '\n')
print(pipeline_updated.groupby('action')[['non_residential_sqft', 'residential_units']].sum())

build    54503
add       4220
Name: action, dtype: int64 

non_residential_sqft    392460056.0
residential_units          439465.0
dtype: float64 

        non_residential_sqft  residential_units
action                                         
add               56993041.0            38754.0
build            335467015.0           400711.0


In [34]:
# drop "geom_id" column
pipeline_updated.drop(['geom_id'], axis=1, inplace=False)
print(list(pipeline_updated))

['development_projects_id', 'OBJECTID', 'raw_id', 'building_name', 'site_name', 'action', 'address', 'city', 'zip', 'county', 'x', 'y', 'geom_id', 'year_built', 'building_type_det', 'building_type', 'building_type_id', 'development_type_id', 'building_sqft', 'non_residential_sqft', 'residential_units', 'unit_ave_sqft', 'tenure', 'rent_type', 'stories', 'parking_spaces', 'average_weighted_rent', 'last_sale_year', 'last_sale_price', 'deed_restricted_units', 'source', 'PARCEL_ID', 'ZONE_ID', 'edit_date', 'editor', 'data_source', 'parcel_id_basis', 'building_id_basis', 'residential_sqft', 'preserved_units', 'inclusionary_units', 'subsidized_units']


In [35]:
# # write out
# pipeline_updated.to_csv(
#     r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\parcels_buildings_agents\development_pipeline_2024-07-31.csv', index=False
# )

### update on 8/13/2024: remove "manual" projects from pipeline (while still making sure the earliest project on a parcel has "build" action)

In [36]:
# pipeline data used in Draft Blueprint modeling

pipeline_dbp_version = pd.read_csv(
    r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\parcels_buildings_agents\development_pipeline_NP_2024-03-08.csv'
)
print('row count: {}'.format(len(pipeline_dbp_version)))
print('\nunique development_projects_id: {}\n'.format(pipeline_dbp_version['development_projects_id'].nunique()))

print(pipeline_dbp_version['source'].value_counts(dropna=False), '\n')
print(pipeline_dbp_version[['non_residential_sqft', 'residential_units']].sum(), '\n')
print(pipeline_dbp_version.groupby('source')[['non_residential_sqft', 'residential_units']].sum())

row count: 58723

unique development_projects_id: 58723

basis_buildingsv0    44797
bas_bp_new            5807
cs                    3105
rf                    2785
basis                 1898
manual                 331
Name: source, dtype: int64 

non_residential_sqft    392460056.0
residential_units          439465.0
dtype: float64 

                   non_residential_sqft  residential_units
source                                                    
bas_bp_new                    7185174.0             8598.0
basis                        49555908.0            75564.0
basis_buildingsv0            54698303.0           108945.0
cs                          225220281.0           170523.0
manual                       55800390.0            73012.0
rf                                  0.0             2823.0


C:\Users\ywang\AppData\Local\Temp\ipykernel_12516\4054561755.py:3: DtypeWarning: Columns (3,4,6,7,8,9,14,22,23,26,27,34) have mixed types. Specify dtype option on import or set low_memory=False.
  pipeline_dbp_version = pd.read_csv(


In [37]:
# drop "manual"
pipeline_wo_manual = pipeline_dbp_version.loc[pipeline_dbp_version['source'] != 'manual']
print(len(pipeline_wo_manual))
print(pipeline_wo_manual['action'].value_counts(dropna=False), '\n')
print(pipeline_wo_manual[['non_residential_sqft', 'residential_units']].sum(), '\n')
print(pipeline_wo_manual.groupby('action')[['non_residential_sqft', 'residential_units']].sum())

# apply the modification on "action"
pipeline_wo_manual_updated = ensure_earliest_proj_build(pipeline_wo_manual)

print(pipeline_wo_manual_updated['action'].value_counts(dropna=False), '\n')
print(pipeline_wo_manual_updated[['non_residential_sqft', 'residential_units']].sum(), '\n')
print(pipeline_wo_manual_updated.groupby('action')[['non_residential_sqft', 'residential_units']].sum())

58392
build    53527
add       4865
Name: action, dtype: int64 

non_residential_sqft    336659666.0
residential_units          366453.0
dtype: float64 

        non_residential_sqft  residential_units
action                                         
add               56923039.0            31624.0
build            279736627.0           334829.0
build    54275
add       4117
Name: action, dtype: int64 

non_residential_sqft    336659666.0
residential_units          366453.0
dtype: float64 

        non_residential_sqft  residential_units
action                                         
add               36660146.0            21370.0
build            299999520.0           345083.0


In [38]:
# drop "geom_id" column
pipeline_wo_manual_updated.drop(['geom_id'], axis=1, inplace=False)
print(list(pipeline_wo_manual_updated))

['development_projects_id', 'OBJECTID', 'raw_id', 'building_name', 'site_name', 'action', 'address', 'city', 'zip', 'county', 'x', 'y', 'geom_id', 'year_built', 'building_type_det', 'building_type', 'building_type_id', 'development_type_id', 'building_sqft', 'non_residential_sqft', 'residential_units', 'unit_ave_sqft', 'tenure', 'rent_type', 'stories', 'parking_spaces', 'average_weighted_rent', 'last_sale_year', 'last_sale_price', 'deed_restricted_units', 'source', 'PARCEL_ID', 'ZONE_ID', 'edit_date', 'editor', 'data_source', 'parcel_id_basis', 'building_id_basis', 'residential_sqft', 'preserved_units', 'inclusionary_units', 'subsidized_units']


In [39]:
# # write out
# pipeline_wo_manual_updated.to_csv(
#     r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\parcels_buildings_agents\development_pipeline_wo_manual_2024-08-13.csv', index=False
# )

### update on 8/15/2024: use PBA50 scen25 (NoProject) equavilent pipeline projects for test run.

In [40]:
pba50_proj = pd.read_csv(
    r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Development Projects\2021_0308_1907_development_projects.csv')
print(len(pba50_proj))

display(pba50_proj.groupby(['scen25']).agg(
    {'non_residential_sqft': 'sum',
     'residential_units': 'sum',
     'source': 'count'}))

display(pba50_proj.groupby(['scen24']).agg(
    {'non_residential_sqft': 'sum',
     'residential_units': 'sum',
     'source': 'count'}))

49665


C:\Users\ywang\AppData\Local\Temp\ipykernel_12516\292447193.py:1: DtypeWarning: Columns (3,4,28,30,44,45,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  pba50_proj = pd.read_csv(


,non_residential_sqft,residential_units,source
scen25,,,
0,154009073,789938,12872
1,337074344,398569,36793


,non_residential_sqft,residential_units,source
scen24,,,
0,55603639,408824,11219
1,435479778,779683,38446


In [41]:
print('NoProject:')
pba50_NP = pba50_proj.loc[pba50_proj['scen25'] == 1]
print(len(pba50_NP))
display(pba50_NP.groupby(['source']).agg(
    {'non_residential_sqft': 'sum',
     'residential_units': 'sum',
     'source': 'count'}))

print('\nFinal Blueprint:')
pba50_FBP = pba50_proj.loc[pba50_proj['scen24'] == 1]
print(len(pba50_FBP))
display(pba50_FBP.groupby(['source']).agg(
    {'non_residential_sqft': 'sum',
     'residential_units': 'sum',
     'source': 'count'}))

NoProject:
36793


,non_residential_sqft,residential_units,source
source,,,
bas_bp_new,15861653,34778,26292
basis,49709367,88440,1968
cs,210648032,182263,3591
manual,60855292,88526,425
rf,0,4562,4517



Final Blueprint:
38446


,non_residential_sqft,residential_units,source
source,,,
bas_bp_new,15861653,34778,26292
basis,49709367,88440,1968
cs,210648032,182263,3591
incubator,11250000,0,25
mall_office,27217094,155602,510
manual,60855292,88526,425
opp,5860364,57942,88
ppa,45701976,0,138
pub,8376000,167570,892


In [42]:
# check if important fields in the PBA50+ schema is missing from the PBA50 data
pba50plus_cols = list(pipeline_dbp_version)
pba50plus_cols.remove('geom_id')
# print(pba50plus_cols)
not_in_pba50 = [i for i in list(pba50plus_cols) if not (i in list(pba50_NP))]
print(not_in_pba50)

['data_source', 'parcel_id_basis', 'building_id_basis', 'residential_sqft', 'preserved_units', 'inclusionary_units', 'subsidized_units']


In [43]:
# check what value should be assigned to the 'data_source' field
pipeline_dbp_version.groupby(['data_source', 'source'])['source'].count()

data_source      source           
basis_buildings  basis_buildingsv0    44797
baus_pipeline    bas_bp_new            5807
                 basis                 1898
                 cs                    3105
                 manual                 331
                 rf                    2785
Name: source, dtype: int64

In [44]:
# keep only needed fields, and add 'data_source'
pba50_NP_pipeline = pba50_NP[[i for i in list(pba50plus_cols) if (i in list(pba50_NP))]]
pba50_NP_pipeline['data_source'] = 'baus_pipeline'

C:\Users\ywang\AppData\Local\Temp\ipykernel_12516\1362611316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pba50_NP_pipeline['data_source'] = 'baus_pipeline'


In [46]:
# write out
print(len(pba50_NP_pipeline))
pba50_NP_pipeline.to_csv(
    r'M:\urban_modeling\baus\BAUS Inputs\basis_inputs\parcels_buildings_agents\development_pipeline_PBA50_NPequiv_2024-08-15.csv', index=False
)

36793
